# MSDS 460 Final Project

Adam Tierney, Leovic Moreno, Sebastian Ricketts, and Michael Christianson  
MSDS 460: Decision Analytics  
Professor Joe Wilck  
December 4, 2022

This notebook will contain the optimization code needed for our final project.

## Code Setup

### Package Preparation

Install and import all modules needed to optimize our problem.

In [ ]:
!pip install PuLP==2.6.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!apt-get install -y -qq glpk-utils

In [ ]:
from google.colab import auth, drive
from itertools import product
import os
import pandas as pd
from pprint import pprint
from pulp import *
import requests

### Google Drive Setup

The following code setup up the connection to Google Drive for inputs and outputs.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


The following cell contains several helper functions for the optimization problem.

In [ ]:
# This function identifies the user of this Colab notebook. This is useful when determining where the data and code for this project are stored.
def get_user_email():
  gcloud_token = !gcloud auth print-access-token
  gcloud_tokeninfo = requests.get('https://www.googleapis.com/oauth2/v3/tokeninfo?access_token=' + gcloud_token[0]).json()
  return gcloud_tokeninfo['email']

The following cell mounts Google drive to the local file system so we can interact with it. 

In [ ]:
drive.mount('/content/drive')
auth.authenticate_user()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Each team member might keep the shared folder for our data and code in a different place inside Google drive. This cell will direct Colab to the correct place based on the user. 

In [ ]:
# Group Access 
# This dictionary is used to map the user to their corresponding Google drive folder that stores their code.
WORKING_DIRECTORY_DICT = {
    "mikedc94@gmail.com": "/content/drive/MyDrive/Education/School Work/Data Science Masters/MSDS 460 - Decision Analytics/MSDS 460 Final Project/Data and Code/",
    "leovicmoreno2023@u.northwestern.edu": "/content/drive/MyDrive/MSDS 460 Final Project/Data and Code/",
    "adamtierney2024@u.northwestern.edu": "/content/drive/MyDrive/MSDS 460 Final Project/Data and Code/",
    "sebastianricketts2025@u.northwestern.edu": "/content/drive/MyDrive/MSDS460/MSDS 460 Final Project/Data and Code/"
}

email = get_user_email()
os.chdir(WORKING_DIRECTORY_DICT[email])
print(f"Working directory set to: {os.getcwd()}")

Working directory set to: /content/drive/MyDrive/Education/School Work/Data Science Masters/MSDS 460 - Decision Analytics/MSDS 460 Final Project/Data and Code


## Problem Setup

### Problem Inputs and Constants


The following cell contains several constants. Most of the constants store inputs from the problem setup with obvious names.

In [ ]:
ITEMS = ['Widgets', 'Gadgets', 'Flugels']
PERIODS = [1, 2, 3, 4, 5]
PLANTS = ['A', 'B']
LABOR_TYPES = ['Regular', 'Overtime']
MATERIAL_TYPES = ['Material 1', 'Material 2']
LABOR_LIMITS = {
    'A': 2500,
    'B': 3800
}
INVENTORY_LIMITS = {
    'A': 70,
    'B': 50
}
MARKETING_COST_PER_EXTRA_UNIT = {
    'Widgets': 160, 
    'Gadgets': 120, 
    'Flugels': 180
}
MATERIALS_LIMITS = {
    'Material 1': 70*2000,
    "Material 2": 2.5*2000
}
SALES_PRICES = {
    'Widgets': 2490,
    'Gadgets': 1990,
    'Flugels': 2970
}
MATERIAL_COSTS = {
    'A': {
        "Material 1": 1.25,
        "Material 2": 2.65
    },
    'B': {
        "Material 1": 1.45,
        "Material 2": 2.9
    }
}

There are several more complicated problem inputs that are easier to store in Excel. We will read those in here so they can be used elsewhere in the problem.

#### Item Costs

In [ ]:
item_costs_df = pd.read_excel("ProblemInputs.xlsx", sheet_name='ITEM_COSTS')
item_costs_df

,item,plant,Material 1,Material 2,labor hours,inventory cost,transportation cost
0,Widgets,A,194,8.6,9.5,7.5,6.3
1,Gadgets,A,230,0.0,7.1,5.5,4.6
2,Flugels,A,178,11.6,11.1,6.5,5.5
3,Widgets,B,188,9.2,9.1,7.8,6.5
4,Gadgets,B,225,0.0,7.8,5.7,5.0
5,Flugels,B,170,10.8,10.6,7.0,5.7


In [ ]:
item_costs_dict = {}
for plant_item_index, cost_dict in item_costs_df.set_index(['plant', 'item']).to_dict('index').items():
  curr_plant, curr_item = plant_item_index
  if curr_plant not in item_costs_dict:
    item_costs_dict[curr_plant] = {}
  item_costs_dict[curr_plant][curr_item] = cost_dict
pprint(item_costs_dict)

{'A': {'Flugels': {'Material 1': 178,
                   'Material 2': 11.6,
                   'inventory cost': 6.5,
                   'labor hours': 11.1,
                   'transportation cost': 5.5},
       'Gadgets': {'Material 1': 230,
                   'Material 2': 0.0,
                   'inventory cost': 5.5,
                   'labor hours': 7.1,
                   'transportation cost': 4.6},
       'Widgets': {'Material 1': 194,
                   'Material 2': 8.6,
                   'inventory cost': 7.5,
                   'labor hours': 9.5,
                   'transportation cost': 6.3}},
 'B': {'Flugels': {'Material 1': 170,
                   'Material 2': 10.8,
                   'inventory cost': 7.0,
                   'labor hours': 10.6,
                   'transportation cost': 5.7},
       'Gadgets': {'Material 1': 225,
                   'Material 2': 0.0,
                   'inventory cost': 5.7,
                   'labor hours': 7.8,
                  

#### Demand Requirements

In [ ]:
demand_requirements_df = pd.read_excel("ProblemInputs.xlsx", sheet_name='DEMAND_REQUIREMENT').melt(id_vars=['item'], var_name='period', value_name='demand')
demand_requirements_df

,item,period,demand
0,Widgets,1,70
1,Gadgets,1,200
2,Flugels,1,140
3,Widgets,2,125
4,Gadgets,2,300
5,Flugels,2,175
6,Widgets,3,185
7,Gadgets,3,295
8,Flugels,3,205
9,Widgets,4,190


In [ ]:
demand_requirements_dict = {}
for item_period_index, demand_dict in demand_requirements_df.set_index(['item', 'period']).to_dict('index').items():
  item, period = item_period_index
  if item not in demand_requirements_dict:
    demand_requirements_dict[item] = {}
  demand_requirements_dict[item][period] = demand_dict['demand']
demand_requirements_dict

{'Widgets': {1: 70, 2: 125, 3: 185, 4: 190, 5: 200},
 'Gadgets': {1: 200, 2: 300, 3: 295, 4: 245, 5: 240},
 'Flugels': {1: 140, 2: 175, 3: 205, 4: 235, 5: 230}}

#### Labor Costs

In [ ]:
labor_costs_df = pd.read_excel("ProblemInputs.xlsx", sheet_name='LABOR_COSTS').round(4)
labor_costs_df

,plant,type,1,2,3,4,5
0,A,Regular,11.0,11.0,11.550,11.550,11.550
1,A,Overtime,16.5,16.5,17.325,17.325,17.325
2,B,Regular,11.0,11.0,12.100,12.100,12.100
3,B,Overtime,16.5,16.5,18.150,18.150,18.150


In [ ]:
labor_costs_dict = {}
for plant_type_index, cost_dict in labor_costs_df.set_index(['plant', 'type']).to_dict('index').items():
  curr_plant, curr_type = plant_type_index
  if curr_plant not in labor_costs_dict:
    labor_costs_dict[curr_plant] = {}
  labor_costs_dict[curr_plant][curr_type] = cost_dict
pprint(labor_costs_dict)

{'A': {'Overtime': {1: 16.5, 2: 16.5, 3: 17.325, 4: 17.325, 5: 17.325},
       'Regular': {1: 11.0, 2: 11.0, 3: 11.55, 4: 11.55, 5: 11.55}},
 'B': {'Overtime': {1: 16.5, 2: 16.5, 3: 18.15, 4: 18.15, 5: 18.15},
       'Regular': {1: 11.0, 2: 11.0, 3: 12.1, 4: 12.1, 5: 12.1}}}


### LP Object Setup

This problem has a lot going on, so we are going to break it down into smaller parts. We are trying to maximize the profit of the manufacturing operation. As an equation, we are solving the following:

$ \text{Revenue} - \text{Expenses}$

We can further break down expenses as:

$ \text{Revenue} - \text{Cost of Marketing} - \text{Cost of Inventory} - \text{Cost of Labor} - \text{Cost of Materials} - \text{Cost of Transportation}$

We will break each part down in it's own section but first, we will set up the problem object in pulp.

In [ ]:
prob = LpProblem("Manufacturing_Optimization", LpMaximize)

## Problem Variables Creation

### Revenue Variables
The only way to generate revenue is to produce and sell items. The transportation variables will be used to calcualte the total revenue, however production variables are still required to ensure that we are make the items we plan to sell.  We will need a total of 30 variables for number of items produced of each kind, in each period, and at each plant ($3 \times 5 \times 2$). 

In [ ]:
production_vars_dict = {}
for curr_plant in PLANTS:
  production_vars_dict[curr_plant] = {}
  for curr_item in ITEMS:
    production_vars_dict[curr_plant][curr_item] = {}
    for curr_period in PERIODS:
      production_vars_dict[curr_plant][curr_item][curr_period] = LpVariable(f"{curr_item} produced at plant {curr_plant} in period {curr_period}", 0, None)
pprint(production_vars_dict)

{'A': {'Flugels': {1: Flugels_produced_at_plant_A_in_period_1,
                   2: Flugels_produced_at_plant_A_in_period_2,
                   3: Flugels_produced_at_plant_A_in_period_3,
                   4: Flugels_produced_at_plant_A_in_period_4,
                   5: Flugels_produced_at_plant_A_in_period_5},
       'Gadgets': {1: Gadgets_produced_at_plant_A_in_period_1,
                   2: Gadgets_produced_at_plant_A_in_period_2,
                   3: Gadgets_produced_at_plant_A_in_period_3,
                   4: Gadgets_produced_at_plant_A_in_period_4,
                   5: Gadgets_produced_at_plant_A_in_period_5},
       'Widgets': {1: Widgets_produced_at_plant_A_in_period_1,
                   2: Widgets_produced_at_plant_A_in_period_2,
                   3: Widgets_produced_at_plant_A_in_period_3,
                   4: Widgets_produced_at_plant_A_in_period_4,
                   5: Widgets_produced_at_plant_A_in_period_5}},
 'B': {'Flugels': {1: Flugels_produced_at_plant_B_i

There are several constraints on revenue including materials, labor, and inventory. We will be dealing with these in the associated expense sections once the expense variables have been defined. There is one other key point to notice about the production variables. It is unclear if an item produced in a period is sold immediately or put into inventory to sell in a future period. One might assume that we need additional varaibles to track the number of items sold. For our setup, we will use the count of items transportated to the distribution center both for transportation expenses and sales.

### Marketing Variables

We will begin with marketing as it is the only expense that can increase revenue instead of simply limiting it. The first insight is that we will only have a marketing budget for the first four periods as marketing in the current period affects demand in the next. Despite the fact that we cannot increase demand in the first period with marketing, we will still create a variable for the increase in demand in the first period and set the upper (and lower) limit of the variable to zero. This makes the code simpler in a few places. Notice that constraint in the variable creation line of the following cell. 

In [ ]:
marketing_vars_dict = {}
for curr_item in ITEMS:
  marketing_vars_dict[curr_item] = {}
  for curr_period in PERIODS:
    marketing_vars_dict[curr_item][curr_period] = LpVariable(f"{curr_item} sold in period {curr_period} from marketing in period {curr_period - 1}", 0, 0 if curr_period == 1 else None)
pprint(marketing_vars_dict)

{'Flugels': {1: Flugels_sold_in_period_1_from_marketing_in_period_0,
             2: Flugels_sold_in_period_2_from_marketing_in_period_1,
             3: Flugels_sold_in_period_3_from_marketing_in_period_2,
             4: Flugels_sold_in_period_4_from_marketing_in_period_3,
             5: Flugels_sold_in_period_5_from_marketing_in_period_4},
 'Gadgets': {1: Gadgets_sold_in_period_1_from_marketing_in_period_0,
             2: Gadgets_sold_in_period_2_from_marketing_in_period_1,
             3: Gadgets_sold_in_period_3_from_marketing_in_period_2,
             4: Gadgets_sold_in_period_4_from_marketing_in_period_3,
             5: Gadgets_sold_in_period_5_from_marketing_in_period_4},
 'Widgets': {1: Widgets_sold_in_period_1_from_marketing_in_period_0,
             2: Widgets_sold_in_period_2_from_marketing_in_period_1,
             3: Widgets_sold_in_period_3_from_marketing_in_period_2,
             4: Widgets_sold_in_period_4_from_marketing_in_period_3,
             5: Widgets_sold_in_

### Transportation Variables

The company must transport their finished products from the plants to the distribution centers. These variables will double as the sales variables. We will need an additional 30 variables. One for each combination of plant, item, and period.

In [ ]:
transportation_vars_dict = {}
for curr_plant in PLANTS:
  transportation_vars_dict[curr_plant] = {}
  for curr_item in ITEMS:
    transportation_vars_dict[curr_plant][curr_item] = {}
    for curr_period in PERIODS:
      transportation_vars_dict[curr_plant][curr_item][curr_period] = LpVariable(f"{curr_item} transported from plant {curr_plant} in period {curr_period}", 0, None)
pprint(transportation_vars_dict)

{'A': {'Flugels': {1: Flugels_transported_from_plant_A_in_period_1,
                   2: Flugels_transported_from_plant_A_in_period_2,
                   3: Flugels_transported_from_plant_A_in_period_3,
                   4: Flugels_transported_from_plant_A_in_period_4,
                   5: Flugels_transported_from_plant_A_in_period_5},
       'Gadgets': {1: Gadgets_transported_from_plant_A_in_period_1,
                   2: Gadgets_transported_from_plant_A_in_period_2,
                   3: Gadgets_transported_from_plant_A_in_period_3,
                   4: Gadgets_transported_from_plant_A_in_period_4,
                   5: Gadgets_transported_from_plant_A_in_period_5},
       'Widgets': {1: Widgets_transported_from_plant_A_in_period_1,
                   2: Widgets_transported_from_plant_A_in_period_2,
                   3: Widgets_transported_from_plant_A_in_period_3,
                   4: Widgets_transported_from_plant_A_in_period_4,
                   5: Widgets_transported_from

### Labor Variables

Labor is measured by plant, period, and by regular hours and overtime hours. We will need 20 total variables for labor.

In [ ]:
labor_vars_dict = {}
for curr_plant in PLANTS:
  labor_vars_dict[curr_plant] = {}
  for curr_period in PERIODS:
    labor_vars_dict[curr_plant][curr_period] = {}
    for curr_labor_type in LABOR_TYPES:
      # some labor hour variables have an upper limit
      if curr_labor_type == "Regular":
        var_upper_limit = LABOR_LIMITS[curr_plant]
      else:
        var_upper_limit = None
      labor_vars_dict[curr_plant][curr_period][curr_labor_type] = LpVariable(f"{curr_labor_type} labor hours in plant {curr_plant} in period {curr_period}", 0, var_upper_limit)
pprint(labor_vars_dict)

{'A': {1: {'Overtime': Overtime_labor_hours_in_plant_A_in_period_1,
           'Regular': Regular_labor_hours_in_plant_A_in_period_1},
       2: {'Overtime': Overtime_labor_hours_in_plant_A_in_period_2,
           'Regular': Regular_labor_hours_in_plant_A_in_period_2},
       3: {'Overtime': Overtime_labor_hours_in_plant_A_in_period_3,
           'Regular': Regular_labor_hours_in_plant_A_in_period_3},
       4: {'Overtime': Overtime_labor_hours_in_plant_A_in_period_4,
           'Regular': Regular_labor_hours_in_plant_A_in_period_4},
       5: {'Overtime': Overtime_labor_hours_in_plant_A_in_period_5,
           'Regular': Regular_labor_hours_in_plant_A_in_period_5}},
 'B': {1: {'Overtime': Overtime_labor_hours_in_plant_B_in_period_1,
           'Regular': Regular_labor_hours_in_plant_B_in_period_1},
       2: {'Overtime': Overtime_labor_hours_in_plant_B_in_period_2,
           'Regular': Regular_labor_hours_in_plant_B_in_period_2},
       3: {'Overtime': Overtime_labor_hours_in_plant_B

### Materials Variables

There are two material inputs into the items, material 1 and material 2, which need to be shipped to each plant. This will require an additional 20 variables.


In [ ]:
materials_vars_dict = {}
for curr_plant in PLANTS:
  materials_vars_dict[curr_plant] = {}
  for curr_period in PERIODS:
    materials_vars_dict[curr_plant][curr_period] = {}
    for curr_material_type in MATERIAL_TYPES:
      materials_vars_dict[curr_plant][curr_period][curr_material_type] = LpVariable(f"Pounds of {curr_material_type} shipped to plant {curr_plant} in period {curr_period}", 0, None)
pprint(materials_vars_dict)

{'A': {1: {'Material 1': Pounds_of_Material_1_shipped_to_plant_A_in_period_1,
           'Material 2': Pounds_of_Material_2_shipped_to_plant_A_in_period_1},
       2: {'Material 1': Pounds_of_Material_1_shipped_to_plant_A_in_period_2,
           'Material 2': Pounds_of_Material_2_shipped_to_plant_A_in_period_2},
       3: {'Material 1': Pounds_of_Material_1_shipped_to_plant_A_in_period_3,
           'Material 2': Pounds_of_Material_2_shipped_to_plant_A_in_period_3},
       4: {'Material 1': Pounds_of_Material_1_shipped_to_plant_A_in_period_4,
           'Material 2': Pounds_of_Material_2_shipped_to_plant_A_in_period_4},
       5: {'Material 1': Pounds_of_Material_1_shipped_to_plant_A_in_period_5,
           'Material 2': Pounds_of_Material_2_shipped_to_plant_A_in_period_5}},
 'B': {1: {'Material 1': Pounds_of_Material_1_shipped_to_plant_B_in_period_1,
           'Material 2': Pounds_of_Material_2_shipped_to_plant_B_in_period_1},
       2: {'Material 1': Pounds_of_Material_1_shipped_to_

### Inventory Variables

We need variables that store the amount of inventory stored at the end of a period for the next period. In the dictionary that stores these variables, I will use the first of the two periods to index the variable. For example, the inventory stored from period 1 to period 2 would be indexed as 1. Despite the fact that there is no inventory at the begining of period 1 and the end of period 5, we will still create accounting inventory variables for these situations and force the variables to be set to zero. This will make the problem setup and interpretation more consistent across periods. 

In [ ]:
inventory_vars_dict = {}
for curr_plant in PLANTS:
  inventory_vars_dict[curr_plant] = {}
  for curr_item in ITEMS:
    inventory_vars_dict[curr_plant][curr_item] = {}
    for curr_period in [0] + PERIODS:
      inventory_vars_dict[curr_plant][curr_item][curr_period] = LpVariable(f"{curr_item} stored from period {curr_period} to {curr_period + 1} at plant {curr_plant}", 0, 0 if curr_period in [0, 5] else None)
pprint(inventory_vars_dict)

{'A': {'Flugels': {0: Flugels_stored_from_period_0_to_1_at_plant_A,
                   1: Flugels_stored_from_period_1_to_2_at_plant_A,
                   2: Flugels_stored_from_period_2_to_3_at_plant_A,
                   3: Flugels_stored_from_period_3_to_4_at_plant_A,
                   4: Flugels_stored_from_period_4_to_5_at_plant_A,
                   5: Flugels_stored_from_period_5_to_6_at_plant_A},
       'Gadgets': {0: Gadgets_stored_from_period_0_to_1_at_plant_A,
                   1: Gadgets_stored_from_period_1_to_2_at_plant_A,
                   2: Gadgets_stored_from_period_2_to_3_at_plant_A,
                   3: Gadgets_stored_from_period_3_to_4_at_plant_A,
                   4: Gadgets_stored_from_period_4_to_5_at_plant_A,
                   5: Gadgets_stored_from_period_5_to_6_at_plant_A},
       'Widgets': {0: Widgets_stored_from_period_0_to_1_at_plant_A,
                   1: Widgets_stored_from_period_1_to_2_at_plant_A,
                   2: Widgets_stored_from_peri

We now have all of the variables that we need to complete this problem. Now we need to defind the constraints, calculate the profit, and solve the problem.

## Constraints Creation

Now that we have all of the variables, it is time to build each of the constraints.

### Marketing Budget Constraint
The only constraint for marketing spending is that it should not exceed $70,000 for the entire planning period. This can be computed by summing all of the marketing spending variables.

As an equation, this contraint is: $\sum_{i \in items, t \in time periods}^{} (\text{Additional units of demand for item i in time period t}) \times (\text{Marketing cost of driving an additional unit of item i}) \le 70000$ 

In [ ]:
prob += LpConstraint(sum([marketing_vars_dict[item][period] * MARKETING_COST_PER_EXTRA_UNIT[item] for item, period in product(ITEMS, PERIODS)]), rhs= 70_000, sense=LpConstraintLE, name='Marketing Budget Constraint')

### Materials Constraints

The materials constraints are quite simple. Enough material must be shipped to each plan for the required production of all items produced in that period. In addition, we must make sure that we do not plan to purchase more than is available from the supplier.

For material 1 and 2, create a constraint that the amount transported to the plant in that period must be greater than or equal to the amount needed for production.

For a particular material, plant, and time period, this constraint is: $\text{Pounds of material shipped the plant} - (\sum_{i \in items}^{} \text{Materials required to produce one item i} \times \text{Units of item i produced at the plant}) \ge 0$ 

In [ ]:
for curr_plant, curr_material, curr_period in product(PLANTS, MATERIAL_TYPES, PERIODS):
  # in the below constraint, the left hand side is the amount of material purchased and shipped to the plant minus the material used for production in that period at that plant.
  material_calc_list = []
  for curr_item in ITEMS:
    material_calc_list.append(item_costs_dict[curr_plant][curr_item][curr_material] * production_vars_dict[curr_plant][curr_item][curr_period])
  prob += LpConstraint(materials_vars_dict[curr_plant][curr_period][curr_material] - sum(material_calc_list), rhs=0, sense=LpConstraintGE, name=f'Plant {curr_plant} {curr_material} requirement in period {curr_period}')

Create the constraint imposed by the limited supply of materials available from the supplier.

For a particular material in a time period, this constraint is: $\text{Pounds of material shipped to plant A } + \text{Pounds of material shipped to plant B } \le \text{The limit for the amount of materials available from the supplier}$

In [ ]:
for curr_material, curr_period in product(MATERIAL_TYPES, PERIODS):
  prob += LpConstraint(materials_vars_dict['A'][curr_period][curr_material] + materials_vars_dict['B'][curr_period][curr_material], rhs=MATERIALS_LIMITS[curr_material], sense=LpConstraintLE, name=f'Purchase limit for {curr_material} in period {curr_period}')

### Labor Constraints

This constraint is very similar to the materials constraint. We need to ensure we have enough labor hours to create the planned production at each plant in each period. 

As an equation, this constraint is for a particular plant in a particular time period: $(\sum_{\forall labor types}^{} \text{The number of labor hours available of a certain type}) - (\sum_{i \in items}^{} \text{Number of items i produced} \times \text{Labor hours required to produce one unit of item i}) \ge 0$ 

In [ ]:
for curr_plant, curr_period in product(PLANTS, PERIODS):
      # in the below constraint, the left hand side is the amount of labor hour available minus the amount of labor hours requred for production, which must be greater than 0
      labor_calc_list = []
      for curr_item in ITEMS:
        labor_calc_list.append(item_costs_dict[curr_plant][curr_item]["labor hours"] * production_vars_dict[curr_plant][curr_item][curr_period])
      prob += LpConstraint(sum(labor_vars_dict[curr_plant][curr_period].values()) - sum(labor_calc_list), rhs=0, sense=LpConstraintGE, name=f'Plant {curr_plant} labor hours requirement in period {curr_period}')

### Inventory Limits

We need to build constraints that any unsold items at the end of the period must be stored in inventory, which has an associated cost and limit. We also need to create constraints that the inventory does not exceed the available space. This could not be done in the variable declaration because the inventory is for the combination of all items. 

For a particular plant in a particular time period, this constraint is:  
$\sum_{items} \text{Number of items in inventory} \le \text{Inventory Limit}$

In [ ]:
for curr_plant, curr_period in product(PLANTS, PERIODS):
  # in the below constraint, the left hand side is the amount number of items in the given period that are carried to the next. The right hand side is the inventory capacity for that plant
  inventory_calc_list = []
  for curr_item in ITEMS:
    inventory_calc_list.append(inventory_vars_dict[curr_plant][curr_item][curr_period])
  prob += LpConstraint(sum(inventory_calc_list), rhs=INVENTORY_LIMITS[curr_plant], sense=LpConstraintLE, name=f'Plant {curr_plant} inventory capcaity limit for items stored from {curr_period} to {curr_period + 1}')

And now set up the equality constraint that $\text{Starting Inventory} + \text{Produced in Period} - \text{Shipped to Distribution} - \text{Ending Inventory} = 0$

In [ ]:
for curr_plant, curr_item, curr_period in product(PLANTS, ITEMS, PERIODS):
  # in the below constraint, the left hand side is the number of items in the given period that are carried to the next. The right hand side is the inventory capacity for that plant
  prob += LpConstraint(
    # inventory from the prior period
    inventory_vars_dict[curr_plant][curr_item][curr_period - 1]
    # plus production in the current period
    + production_vars_dict[curr_plant][curr_item][curr_period]
    # minus items shipped for sale in the current period
    - transportation_vars_dict[curr_plant][curr_item][curr_period]
    # minus the inventory at the end of the period
    - inventory_vars_dict[curr_plant][curr_item][curr_period]
  , rhs=0, sense=LpConstraintEQ, name=f'Plant {curr_plant} period {curr_period} inventory transportation and production logical requirement for {curr_item}')

### Production Constraints

Most of the above constraints apply to production variables, so most of the production constraints are already set. For example, we already know that we will not plan production above the availability of labor, inventory, or materials. We still need to set the limits of production caused by demand. I am making an assumption here that we must meet demand, even if the final widget produced is unprofitable. Due to this, I am going to create equality constraints that calculate the demand given the already constracted purchases and the extra demand from marketing. To force the inventory and production to be adequate for sales, I will set shipments equal to demand.

As an equation, in a particular time period, and for a particular item, this constraint is:   
$\text{Items transported to distribution from plant A} + \text{Items transported to distribution from plant B} - \text{Base demand} - \text{Demand from marketing} = 0$

In [ ]:
for curr_item, curr_period in product(ITEMS, PERIODS):
  # Left side is shipments to distribution minus the demand from presales and marketing, which should equal 0
  prob += LpConstraint(
              # The number of items sold from plant A in the period
              transportation_vars_dict['A'][curr_item][curr_period]
              # the number of items sold from plant B in the period 
              + transportation_vars_dict['B'][curr_item][curr_period] 
              # the demand that is already anticipated outside marketing
              - demand_requirements_dict[curr_item][curr_period] 
              # the demand from marketing in the previous period
              - marketing_vars_dict[curr_item][curr_period]
          , rhs=0, sense=LpConstraintEQ, name=f'Demand for {curr_item} in period {curr_period}')


We now have all of the constraints needed to optomize the problem.

## Objection Function Creation

The objective function for this problem will be quite complicated.  We are going to create a list of terms that will eventually all be summed together in the problem objective function creation.

The equation boils down to:

$ \text{Revenue} - \text{Cost of Marketing} - \text{Cost of Inventory} - \text{Cost of Labor} - \text{Cost of Materials} - \text{Cost of Transportation}$

In [ ]:
objective_function_terms = []

### Revenue

Revenue can be calculated using the number of items shipped to the distribution center in each period.

In [ ]:
for curr_plant, curr_item, curr_period in product(PLANTS, ITEMS, PERIODS):
  # the following line is the number of items sent to distribution (and therefore sold) times the sales price for that item
  objective_function_terms.append(transportation_vars_dict[curr_plant][curr_item][curr_period] * SALES_PRICES[curr_item])

### Cost of Marketing

The cost of marketing is quite straight forward to calculate. We already have the number of item sales driven by marketing and the cost of driving a sale for each item. Notice that from here on out we are subtracting the amounts as these are expenses.

In [ ]:
for curr_item, curr_period in product(ITEMS, PERIODS):
  objective_function_terms.append(-1 * marketing_vars_dict[curr_item][curr_period] * MARKETING_COST_PER_EXTRA_UNIT[item])

### Cost of Inventory

Inventory has a cost in each facility, which must be subtracted from revenue.

In [ ]:
for curr_plant, curr_item, curr_period in product(PLANTS, ITEMS, PERIODS):
  objective_function_terms.append(-1 * item_costs_dict[curr_plant][curr_item]['inventory cost'] * inventory_vars_dict[curr_plant][curr_item][curr_period])

### Cost of Labor

The cost of labor is the number of each labor hours used times the cost of that labor type. The labor costs are stored in the input Excel file.

Now calculate the labor cost of production.

In [ ]:
for curr_plant, curr_period, curr_type in product(PLANTS, PERIODS, LABOR_TYPES):
  objective_function_terms.append(-1 * labor_costs_dict[curr_plant][curr_type][curr_period] * labor_vars_dict[curr_plant][curr_period][curr_type]) 

### Cost of Materials

Calculate the cost of materials purchased and shipped to the plants.

In [ ]:
for curr_plant, curr_period, curr_material in product(PLANTS, PERIODS, MATERIAL_TYPES):
  objective_function_terms.append(-1 * MATERIAL_COSTS[curr_plant][curr_material] * materials_vars_dict[curr_plant][curr_period][curr_material]) 

### Cost of Transportation

We need to calculate the cost of transporting the finished product to the distribution center for each plant and period.

In [ ]:
for curr_plant, curr_item, curr_period in product(PLANTS, ITEMS, PERIODS):
  objective_function_terms.append(-1 * item_costs_dict[curr_plant][curr_item]['transportation cost'] * transportation_vars_dict[curr_plant][curr_item][curr_period])

The objective function now has all of it's terms. We are ready to combine them into the actual function.

In [ ]:
prob += sum(objective_function_terms)

## Optimization

We finally have all variables, constraints, and the objective function set up. We are ready to optimize.

In [ ]:
prob.writeLP("Manufacturing_Optimization.lp")
status = prob.solve(GLPK(options=['--ranges', 'manufacturing_plan_sensitivity.sen']))
print(LpStatus[status])
assert(LpStatus[status] == "Optimal")

Optimal


### Output

Now that we have an optimized problem, output the optimal solution

In [ ]:
# print the results
print("Optimal Operations Plan:\n")
for curr_var in [x for x in prob.variables() if x.name != "__dummy"]:
    print(f"{curr_var.name.replace('_', ' ')}: {round(curr_var.varValue, 1)}")

print("\n" + f"Maximum Profit: ${'{:,.2f}'.format(value(prob.objective))}")

Optimal Operations Plan:

Flugels produced at plant A in period 1: 0.0
Flugels produced at plant A in period 2: 0.0
Flugels produced at plant A in period 3: 0.0
Flugels produced at plant A in period 4: 0.0
Flugels produced at plant A in period 5: 0.0
Flugels produced at plant B in period 1: 140.0
Flugels produced at plant B in period 2: 356.5
Flugels produced at plant B in period 3: 301.9
Flugels produced at plant B in period 4: 279.1
Flugels produced at plant B in period 5: 274.7
Flugels sold in period 1 from marketing in period 0: 0.0
Flugels sold in period 2 from marketing in period 1: 181.5
Flugels sold in period 3 from marketing in period 2: 96.9
Flugels sold in period 4 from marketing in period 3: 44.1
Flugels sold in period 5 from marketing in period 4: 44.7
Flugels stored from period 0 to 1 at plant A: 0.0
Flugels stored from period 0 to 1 at plant B: 0.0
Flugels stored from period 1 to 2 at plant A: 0.0
Flugels stored from period 1 to 2 at plant B: 0.0
Flugels stored from peri

In [ ]:
def flatten_dict(input_dict, prefix_index=()):
  output_dict = {}
  for curr_key, curr_val in input_dict.items():
    if type(curr_val) == dict:
      output_dict.update(flatten_dict(curr_val, prefix_index=prefix_index + (curr_key,)))
    else:
      output_dict[prefix_index + (curr_key,)] = curr_val
  return output_dict

def save_table(var_dict, key_labels, column_index, sheet_name, excel_writer, round_to=None):
  reform = {key: {'value': value(val)} for key, val in flatten_dict(var_dict).items()}
  df = pd.DataFrame().from_dict(reform).T.sort_index()
  df.index.names = key_labels
  df = pd.pivot_table(df, index=key_labels[:column_index] + key_labels[column_index + 1:], columns=[key_labels[column_index]], values='value', aggfunc='first')
  df = df.reset_index()
  df.round(round_to).to_excel(excel_writer, sheet_name=sheet_name, index=False)
  print(sheet_name)
  print(df)
  print()
  return df
  
# Save each varible type to it's own tab in an excel workbook.
with pd.ExcelWriter("ProblemOutput.xlsx") as writer:
  save_table(production_vars_dict,     ('facility', 'item', 'period'),           
                2, "PRODUCTION", writer, 2)
  save_table(marketing_vars_dict,      ('item', 'period'),                      
                1, "MARKETING", writer, 2)
  save_table(transportation_vars_dict, ('facility', 'item', 'period'),          
                2, "TRANSPORTATION", writer, 2)
  save_table(labor_vars_dict,          ('facility', 'period', 'labor_type'),    
                1, "LABOR", writer, 2)
  save_table(materials_vars_dict,      ('facility', 'period', 'material_type'), 
                1, "MATERIALS", writer, 2)
  save_table(inventory_vars_dict,      ('facility', 'item', 'period'),          
                2, "INVENTORY", writer, 2)


PRODUCTION
period facility     item          1        2         3         4         5
0             A  Flugels    0.00000    0.000    0.0000    0.0000    0.0000
1             A  Gadgets  270.00000  243.035  231.9650  245.0000  240.0000
2             A  Widgets   61.36840    0.000   89.7949   80.0526   83.7895
3             B  Flugels  140.00000  356.481  301.9380  279.1160  274.6900
4             B  Gadgets   50.00000    0.000    0.0000    0.0000    0.0000
5             B  Widgets    8.63158  125.000   95.2051  109.9470  116.2110

MARKETING
period     item    1        2        3        4        5
0       Flugels  0.0  181.481  96.9375  44.1158  44.6898
1       Gadgets  0.0    0.000   0.0000   0.0000   0.0000
2       Widgets  0.0    0.000   0.0000   0.0000   0.0000

TRANSPORTATION
period facility     item          1        2         3         4         5
0             A  Flugels    0.00000    0.000    0.0000    0.0000    0.0000
1             A  Gadgets  200.00000  250.000  295.0000  245